# Gensim LDA

Adapted from https://towardsdatascience.com/topic-modeling-and-latent-dirichlet-allocation-in-python-9bf156893c24

In [126]:
import pandas as pd

csv_file = 'npr.csv'            # the input csv file for training
test_csv_file = 'money.csv'    # contains the transcript
topic_file = 'gensim_topics_'+csv_file

data_path = 'data/'+csv_file
test_path = 'data/'+test_csv_file
topic_path = 'topics/'+topic_file

df = pd.read_csv(data_path);
# df = df[['headline_text']]
df = df[['Article']]
df['index'] = df.index
documents = df


In [71]:
print(len(documents))
print(documents[:5])

11992
                                             Article  index
0  In the Washington of 2016, even when the polic...      0
1    Donald Trump has used Twitter  —   his prefe...      1
2    Donald Trump is unabashedly praising Russian...      2
3  Updated at 2:50 p. m. ET, Russian President Vl...      3
4  From photography, illustration and video, to d...      4


In [72]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(2018)
import nltk
nltk.download('wordnet')
stemmer = SnowballStemmer("english")


[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/bansharee/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [73]:
# words in third person are changed to first person and verbs in past and future tenses are changed into present
# words are reduced to their root form
def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))

def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result

In [74]:
# displaying how preprocessing works
doc_sample = documents[documents['index'] == 4310].values[0][0]
print('original document: ')
words = []
for word in doc_sample.split(' '):
    words.append(word)
print(words)
print('\n\n tokenized and lemmatized document: ')
print(preprocess(doc_sample))

original document: 
['Oklahoma', 'City', 'residents', 'woke', 'early', 'New', 'Year’s', 'Day', 'to', 'a', 'magnitude', '4.', '2', 'quake.', 'Earlier', 'this', 'week,', 'a', 'magnitude', '4.', '3', 'quake', 'struck', 'the', 'same', 'area.', 'The', 'state', 'isn’t', 'historically', 'known', 'for', 'earthquakes,', 'but', 'NPR’s', 'Nell', 'Greenfieldboyce', 'told', 'our', 'Newscast', 'unit', 'that', 'Oklahoma', '”has', 'recently', 'seen', 'a', 'dramatic', 'rise', 'in', 'seismic', 'activity.”', 'Here’s', 'more:', '”If', 'you', 'think', 'of', 'a', 'U.', 'S.', 'state', 'associated', 'with', 'earthquakes,', 'it’s', 'probably', 'California.', 'But', 'really,', 'you', 'should', 'think', 'Oklahoma.', 'In', '2015,', 'Oklahoma', 'hit', 'an', '', '', 'high,', 'with', 'more', 'than', '800', 'quakes', 'of', 'magnitude', '3', 'or', 'greater.', 'That', 'busts', 'the', 'record', 'set', 'in', '2014,', 'which', 'topped', 'the', 'previous', 'record', 'set', 'the', 'year', 'before.', 'State', 'officials', 'h

In [75]:
# preprocess 'headline_text' text from training set
# processed_docs = documents['headline_text'].map(preprocess)
processed_docs = documents['Article'].map(preprocess)
processed_docs[:10]

0    [washington, polici, bipartisan, polit, sens, ...
1    [donald, trump, twitter, prefer, mean, communi...
2    [donald, trump, unabash, prais, russian, presi...
3    [updat, russian, presid, vladimir, putin, say,...
4    [photographi, illustr, video, data, visual, im...
5    [want, join, yoga, class, hat, beatif, instruc...
6    [public, support, debunk, claim, vaccin, caus,...
7    [stand, airport, exit, debat, snack, young, ro...
8    [movi, tri, realist, summon, batman, shouldn, ...
9    [eighteen, year, year, david, fisher, visit, f...
Name: Article, dtype: object

In [76]:
# creating dictionary using words in the training set, mapped to how many times the word appears in the set
dictionary = gensim.corpora.Dictionary(processed_docs)
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break

0 abil
1 accept
2 account
3 act
4 action
5 actual
6 add
7 administr
8 advis
9 affair
10 afloat


In [77]:
# filtering out:
#   * less than 15 documents (absolute number) or
#   * more than 0.5 documents (fraction of total corpus size, not absolute number)
#   * after the above two steps, keep only the first 100000 most frequent tokens

dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)


In [78]:
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]
# bow_corpus[0]

# bow_corpus = []
# for doc in processed_docs:
#     bow = dictionary.doc2bow(doc)
#     bow_corpus.append(bow)


In [79]:
bow_doc_4310 = bow_corpus[4310]

for i in range(len(bow_doc_4310)):
    print("Word {} (\"{}\") appears {} time.".format(
                                                    bow_doc_4310[i][0], 
                                                    dictionary[bow_doc_4310[i][0]], 
                                                    bow_doc_4310[i][1]))

Word 8 ("advis") appears 1 time.
Word 51 ("citi") appears 1 time.
Word 61 ("compromis") appears 1 time.
Word 63 ("concern") appears 1 time.
Word 105 ("econom") appears 1 time.
Word 140 ("greater") appears 1 time.
Word 158 ("includ") appears 1 time.
Word 166 ("issu") appears 1 time.
Word 179 ("look") appears 1 time.
Word 195 ("nation") appears 1 time.
Word 206 ("offici") appears 1 time.
Word 247 ("recent") appears 1 time.
Word 259 ("respons") appears 1 time.
Word 298 ("suggest") appears 1 time.
Word 314 ("tri") appears 1 time.
Word 323 ("unit") appears 1 time.
Word 334 ("week") appears 1 time.
Word 357 ("compani") appears 2 time.
Word 362 ("council") appears 2 time.
Word 374 ("earlier") appears 1 time.
Word 406 ("line") appears 1 time.
Word 451 ("state") appears 3 time.
Word 497 ("earli") appears 1 time.
Word 515 ("increas") appears 1 time.
Word 569 ("unlik") appears 1 time.
Word 572 ("wake") appears 1 time.
Word 615 ("specif") appears 1 time.
Word 636 ("area") appears 1 time.
Word 694 

In [80]:
# TF-IDF weights words based on how often they appear in a document 
# versus how often they appear in the entire corpus
# this helps LDA distinguish topics by weighting more important words higher

from gensim import corpora, models
tfidf = models.TfidfModel(bow_corpus)
corpus_tfidf = tfidf[bow_corpus]
from pprint import pprint

# preview of how this works
pprint(corpus_tfidf[0])
for i in range(len(bow_corpus[0])):
    print(dictionary[bow_corpus[0][i][0]], corpus_tfidf[0][i][1])

[(0, 0.023067990364550667),
 (1, 0.02138290697692375),
 (2, 0.019120276543562995),
 (3, 0.023312189215803015),
 (4, 0.07177692380940279),
 (5, 0.012376338229759715),
 (6, 0.012652765926049652),
 (7, 0.015681833239163178),
 (8, 0.022752553413297974),
 (9, 0.027595984325063022),
 (10, 0.05605640784415791),
 (11, 0.055234263426146644),
 (12, 0.029834588795734765),
 (13, 0.03757301028755038),
 (14, 0.04175927604924723),
 (15, 0.046873736806329634),
 (16, 0.028261189971447376),
 (17, 0.032253677145107414),
 (18, 0.015158469095917521),
 (19, 0.009910816499937069),
 (20, 0.06389342674765507),
 (21, 0.04649554154969994),
 (22, 0.044285416005865226),
 (23, 0.027833764343638497),
 (24, 0.023250503546463713),
 (25, 0.03779800986995305),
 (26, 0.04946093135473547),
 (27, 0.0786047603506468),
 (28, 0.02922961188090144),
 (29, 0.02138290697692375),
 (30, 0.03995684959520006),
 (31, 0.023573953955485928),
 (32, 0.0321408155994278),
 (33, 0.013108675965096848),
 (34, 0.04804888404574625),
 (35, 0.0266

In [81]:
# TODO: HDP goes here for num_topics
# see https://medium.com/analytics-vidhya/text-classification-using-lda-35d5b98d4f05 for HDP 

In [82]:
# training the model using the bow corpus

lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=10, id2word=dictionary, passes=2, workers=2)
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} Words: {}'.format(idx, topic))

Topic: 0 Words: 0.006*"thing" + 0.006*"want" + 0.004*"feel" + 0.004*"live" + 0.004*"write" + 0.004*"book" + 0.004*"stori" + 0.004*"women" + 0.003*"life" + 0.003*"look"
Topic: 1 Words: 0.007*"food" + 0.004*"famili" + 0.004*"want" + 0.004*"look" + 0.003*"live" + 0.003*"help" + 0.003*"need" + 0.003*"world" + 0.003*"start" + 0.003*"thing"
Topic: 2 Words: 0.019*"trump" + 0.007*"clinton" + 0.005*"presid" + 0.005*"campaign" + 0.005*"polit" + 0.005*"state" + 0.005*"news" + 0.004*"candid" + 0.004*"report" + 0.004*"countri"
Topic: 3 Words: 0.012*"health" + 0.007*"care" + 0.006*"patient" + 0.005*"state" + 0.005*"insur" + 0.005*"medic" + 0.005*"research" + 0.004*"studi" + 0.004*"hospit" + 0.004*"need"
Topic: 4 Words: 0.007*"report" + 0.006*"attack" + 0.004*"kill" + 0.003*"state" + 0.003*"forc" + 0.003*"call" + 0.003*"countri" + 0.003*"world" + 0.003*"live" + 0.003*"group"
Topic: 5 Words: 0.012*"school" + 0.010*"student" + 0.007*"state" + 0.005*"educ" + 0.004*"report" + 0.004*"famili" + 0.003*"help

In [83]:
# training the model using the bow corpus

lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, num_topics=10, id2word=dictionary, passes=2, workers=4)
for idx, topic in lda_model_tfidf.print_topics(-1):
    print('Topic: {} Word: {}'.format(idx, topic))

Topic: 0 Word: 0.004*"trump" + 0.002*"clinton" + 0.001*"presid" + 0.001*"women" + 0.001*"report" + 0.001*"obama" + 0.001*"state" + 0.001*"campaign" + 0.001*"student" + 0.001*"elect"
Topic: 1 Word: 0.006*"trump" + 0.002*"republican" + 0.002*"clinton" + 0.002*"elect" + 0.002*"presid" + 0.002*"vote" + 0.002*"democrat" + 0.002*"campaign" + 0.001*"parti" + 0.001*"white"
Topic: 2 Word: 0.002*"trump" + 0.002*"music" + 0.001*"song" + 0.001*"republican" + 0.001*"album" + 0.001*"democrat" + 0.001*"health" + 0.001*"school" + 0.001*"food" + 0.001*"vote"
Topic: 3 Word: 0.004*"trump" + 0.002*"polic" + 0.002*"clinton" + 0.001*"music" + 0.001*"presid" + 0.001*"state" + 0.001*"report" + 0.001*"elect" + 0.001*"campaign" + 0.001*"obama"
Topic: 4 Word: 0.002*"refuge" + 0.002*"trump" + 0.001*"syrian" + 0.001*"elect" + 0.001*"aleppo" + 0.001*"women" + 0.001*"citi" + 0.001*"isi" + 0.001*"water" + 0.001*"school"
Topic: 5 Word: 0.004*"trump" + 0.002*"clinton" + 0.001*"women" + 0.001*"vote" + 0.001*"campaign" +

In [84]:
# check how part of the training set is classified
# first appearing topic is the one assigned to it
print(processed_docs[4310])
for index, score in sorted(lda_model[bow_corpus[4310]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model.print_topic(index, 5)))

['oklahoma', 'citi', 'resid', 'wake', 'earli', 'year', 'magnitud', 'quak', 'earlier', 'week', 'magnitud', 'quak', 'strike', 'area', 'state', 'histor', 'know', 'earthquak', 'nell', 'greenfieldboyc', 'tell', 'newscast', 'unit', 'oklahoma', 'recent', 'see', 'dramat', 'rise', 'seismic', 'activ', 'think', 'state', 'associ', 'earthquak', 'probabl', 'california', 'think', 'oklahoma', 'oklahoma', 'high', 'quak', 'magnitud', 'greater', 'bust', 'record', 'top', 'previous', 'record', 'year', 'state', 'offici', 'say', 'rise', 'unlik', 'repres', 'natur', 'occur', 'process', 'concern', 'quak', 'link', 'drill', 'specif', 'wastewat', 'produc', 'drill', 'pump', 'deep', 'underground', 'dispos', 'well', 'oklahoma', 'tri', 'address', 'issu', 'coordin', 'council', 'seismic', 'activ', 'includ', 'regul', 'scientist', 'industri', 'repres', 'wertz', 'stateimpact', 'oklahoma', 'explain', 'connect', 'industri', 'increas', 'number', 'quak', 'weekend', 'edit', 'saturday', 'novemb', 'product', 'creat', 'toxic', 'wa

In [85]:
# evaluate the tfidf version
for index, score in sorted(lda_model_tfidf[bow_corpus[4310]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model_tfidf.print_topic(index, 5)))


Score: 0.9935162663459778	 
Topic: 0.005*"trump" + 0.003*"clinton" + 0.002*"vote" + 0.002*"elect" + 0.002*"state"


In [86]:
unseen_document = 'if you love traveling, you\'ll love this'
bow_vector = dictionary.doc2bow(preprocess(unseen_document))
for index, score in sorted(lda_model_tfidf[bow_vector], key=lambda tup: -1*tup[1]):
    topic = lda_model_tfidf.print_topic(index, 10)
    print("Score: {}\t Topic words:{}".format(score, re.sub('[^A-Za-z]+', ',', topic)))


Score: 0.7748991847038269	 Topic words:,refuge,trump,syrian,elect,aleppo,women,citi,isi,water,school,
Score: 0.025012578815221786	 Topic words:,trump,clinton,presid,women,report,obama,state,campaign,student,elect,
Score: 0.02501245029270649	 Topic words:,trump,school,vote,book,clinton,women,report,song,polic,compani,
Score: 0.02501240000128746	 Topic words:,trump,polic,clinton,music,presid,state,report,elect,campaign,obama,
Score: 0.025011777877807617	 Topic words:,trump,republican,clinton,elect,presid,vote,democrat,campaign,parti,white,
Score: 0.025011513382196426	 Topic words:,trump,clinton,women,vote,campaign,presid,compani,drug,report,elect,
Score: 0.025010699406266212	 Topic words:,trump,music,song,republican,album,democrat,health,school,food,vote,
Score: 0.025010336190462112	 Topic words:,patient,health,food,clinton,studi,diseas,trump,vote,drug,elect,
Score: 0.02500956691801548	 Topic words:,trump,clinton,vote,elect,state,percent,democrat,obama,voter,compani,
Score: 0.02500944957

In [87]:
# adapted from https://towardsdatascience.com/unsupervised-nlp-topic-models-as-a-supervised-learning-input-cf8ee9e5cf28
train_vecs = []
top_topics = (
    lda_model_tfidf.get_document_topics(bow_vector,
                                    minimum_probability=0.0)
)
pprint(top_topics)

pprint(sorted(lda_model_tfidf[bow_vector], key=lambda tup: -1*tup[1]))

# probabilities for topics 0-num_topics, unsorted
topic_vec = [top_topics[i][1] for i in range(10)]
train_vecs.append(topic_vec)
print(train_vecs)


[(0, 0.02501382),
 (1, 0.025011782),
 (2, 0.025010703),
 (3, 0.025012404),
 (4, 0.7748982),
 (5, 0.025011558),
 (6, 0.025009451),
 (7, 0.02501034),
 (8, 0.025009569),
 (9, 0.025012176)]
[(4, 0.77489895),
 (0, 0.025013052),
 (3, 0.025012452),
 (9, 0.025012178),
 (1, 0.02501178),
 (5, 0.025011515),
 (2, 0.025010703),
 (7, 0.02501034),
 (8, 0.025009569),
 (6, 0.025009451)]
[[0.02501382, 0.025011782, 0.025010703, 0.025012404, 0.7748982, 0.025011558, 0.025009451, 0.02501034, 0.025009569, 0.025012176]]


In [88]:
# getting the topic number of most likely topic
# topic = max(top_topics, key=lambda x: x[1])[0]
# topic

for doc in train_vecs:
    topic = doc.index(max(doc))
    print(topic)

4


In [113]:
# unseen_document = 'if you love traveling, you\'ll love this'
# bow_vector = dictionary.doc2bow(preprocess(unseen_document))
# for index, score in sorted(lda_model_tfidf[bow_vector], key=lambda tup: -1*tup[1]):
#     topic = lda_model_tfidf.print_topic(index, 10)
#     print("Score: {}\t Topic words:{}".format(score, re.sub('[^A-Za-z]+', ',', topic)))
unseen_df = pd.read_csv(test_path)

train_vecs = []
for text_entry in unseen_df['text']:
    bow_vector = dictionary.doc2bow(preprocess(text_entry))
    top_topics = (
        lda_model_tfidf.get_document_topics(bow_vector,
                                        minimum_probability=0.0)
    )

    # probabilities for topics 0-num_topics, unsorted
    train_vecs.append(top_topics)
    

pprint(train_vecs)
    

[[(0, 0.016676301),
  (1, 0.016675584),
  (2, 0.016675625),
  (3, 0.016676933),
  (4, 0.016675146),
  (5, 0.016675454),
  (6, 0.016675713),
  (7, 0.01667541),
  (8, 0.01667971),
  (9, 0.84991413)],
 [(0, 0.05001143),
  (1, 0.050009552),
  (2, 0.050009344),
  (3, 0.050015822),
  (4, 0.050009534),
  (5, 0.05000968),
  (6, 0.05001002),
  (7, 0.05001103),
  (8, 0.5499015),
  (9, 0.050012108)],
 [(0, 0.77461696),
  (1, 0.025041716),
  (2, 0.025045648),
  (3, 0.025043225),
  (4, 0.025040653),
  (5, 0.025043216),
  (6, 0.025041474),
  (7, 0.02504237),
  (8, 0.025042715),
  (9, 0.025042)],
 [(0, 0.01668115),
  (1, 0.01667967),
  (2, 0.01668101),
  (3, 0.016682327),
  (4, 0.01668101),
  (5, 0.016680382),
  (6, 0.016679844),
  (7, 0.01667999),
  (8, 0.6889783),
  (9, 0.17757636)],
 [(0, 0.013173583),
  (1, 0.013172207),
  (2, 0.17420231),
  (3, 0.013173521),
  (4, 0.01317307),
  (5, 0.013172281),
  (6, 0.013173129),
  (7, 0.013172275),
  (8, 0.013176233),
  (9, 0.72041136)],
 [(0, 0.020014076),


In [125]:
topic_results = []
for doc in train_vecs:
    topic_idx, score = max(doc, key=lambda x: x[1]) # numerical topic
    topic_results.append(topic_idx)
    # topic = lda_model_tfidf.print_topic(topic_idx, 10)      # topic words
    # print(re.sub('[^A-Za-z]+', ',', topic))                 # top words in this topic

unseen_df['topics'] = topic_results
unseen_df.head()


,text,topics
0,"Let’s say I had an apple, and you had an orang...",9
1,an apple.,8
2,"We could trade, and both of us would end up ha...",0
3,"But now let’s say I had an apple tree, and you...",8
4,"the apple tree were ready, but the oranges wou...",9


In [128]:
unseen_df.to_csv(topic_path, sep=',', index=False, encoding='utf-8')